In [5]:
import numpy as np

%load_ext autoreload
%autoreload 2
import my_baselines

In [6]:
with open('train.json') as f:
    data = f.readlines()

import ast
data = [ast.literal_eval(x) for x in data]

In [ ]:
data[1]

In [ ]:
def ratingData(data):
    [(1., d['itemID'], d['reviewerID'], )]

In [3]:
from io import open
import unicodedata
import string
import random
import re

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [10]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


MAX_LENGTH = 15

def filterSummaries(summaries):
    return [s for s in summaries if len(s)<MAX_LENGTH]

In [8]:
summaries = [[normalizeString(s) for s in d['summary'].split()] for d in data]
lang = Lang('Summary')
len(summaries)

In [9]:
summaries = filterSummaries(summaries)
len(summaries)

198939